In [5]:
import pandas as pd
import psycopg2

conn = psycopg2.connect(
host="localhost",
database="bank_reviews",
user="admin",
password="admin123",
port=5432
)
cur = conn.cursor()
print("Connected to the database successfully.")

Connected to the database successfully.


In [2]:
import pandas as pd
import psycopg2
from psycopg2 import extras

# Define a Helper Function to Run SQL and Return DataFrame
def run_query(sql, conn):
    """Executes a SQL query and returns the result as a pandas DataFrame."""
    # Note: The UserWarning about the connection object is normal when using raw psycopg2 with pandas.
    return pd.read_sql(sql, conn)

conn = None
try:
    # 1. Connect to PostgreSQL
    conn = psycopg2.connect(
        host="localhost",
        database="bank_reviews",
        user="admin",
        password="admin123",
        port=5432
    )

    # --- 1. Count Reviews per Bank ---
    print("\n--- 1. Count Reviews per Bank ---")
    sql_count_reviews = """
    SELECT 
        b.bank_name, 
        COUNT(r.review_id) AS total_reviews
    FROM 
        reviews r
    JOIN 
        banks b ON b.bank_code = r.bank_code
    GROUP BY 
        b.bank_name
    ORDER BY
        total_reviews DESC;
    """
    df_count = run_query(sql_count_reviews, conn)
    # Changed to .to_string() to avoid the 'tabulate' dependency
    print(df_count.to_string(index=False)) 
    
    # --- 2. Average Rating per Bank ---
    print("\n--- 2. Average Rating per Bank ---")
    sql_avg_rating = """
    SELECT 
        b.bank_name, 
        ROUND(AVG(r.rating)::numeric, 2) AS average_rating,
        COUNT(r.review_id) AS review_count
    FROM 
        reviews r
    JOIN 
        banks b ON b.bank_code = r.bank_code
    GROUP BY 
        b.bank_name
    ORDER BY
        average_rating DESC;
    """
    df_avg = run_query(sql_avg_rating, conn)
    # Changed to .to_string() to avoid the 'tabulate' dependency
    print(df_avg.to_string(index=False))
    
    # --- 3. Count Reviews by Sentiment Label ---
    print("\n--- 3. Count Reviews by Sentiment Label ---")
    sql_sentiment_count = """
    SELECT 
        sentiment_label, 
        COUNT(*) AS count_of_reviews
    FROM 
        reviews
    GROUP BY 
        sentiment_label
    ORDER BY 
        count_of_reviews DESC;
    """
    df_sentiment = run_query(sql_sentiment_count, conn)
    # Changed to .to_string() to avoid the 'tabulate' dependency
    print(df_sentiment.to_string(index=False))

except psycopg2.Error as e:
    print(f"A PostgreSQL error occurred: {e}")
except Exception as e:
    # Catch any remaining errors, including the potential Missing optional dependency error
    print(f"An unexpected error occurred: {e}")
finally:
    if conn:
        conn.close()
        print("\nDatabase connection closed.")


--- 1. Count Reviews per Bank ---
                  bank_name  total_reviews
Commercial Bank of Ethiopia            400
                Dashen Bank            400
          Bank of Abyssinia            400

--- 2. Average Rating per Bank ---
                  bank_name  average_rating  review_count
Commercial Bank of Ethiopia            4.13           400
                Dashen Bank            3.96           400
          Bank of Abyssinia            3.38           400

--- 3. Count Reviews by Sentiment Label ---
sentiment_label  count_of_reviews
        neutral               578
       positive               495
       negative               127

Database connection closed.


C:\Users\ruths\AppData\Local\Temp\ipykernel_33540\1421537311.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql, conn)
C:\Users\ruths\AppData\Local\Temp\ipykernel_33540\1421537311.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql, conn)
C:\Users\ruths\AppData\Local\Temp\ipykernel_33540\1421537311.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql, conn)
